In [1]:
from junifer.storage import HDF5FeatureStorage
from julearn.api import run_cross_validation
from julearn.pipeline import PipelineCreator
from julearn.viz import plot_scores
from julearn.stats.corrected_ttest import corrected_ttest
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import numpy as np

It is highly recommended to configure Git before using DataLad. Set both 'user.name' and 'user.email' configuration variables.


In [2]:
with pd.HDFStore('/home/hsreekri/Julearn_predictions/data/IXI_HistogramGMVdata.h5', mode='r') as store:
    df_hists = store['histograms'] 
    df_bins = store['bins']
    
df_hists.index.name = 'subject'
df_hists = df_hists.reset_index()


In [3]:
df_demographics = pd.read_csv('/home/hsreekri/Julearn_predictions/data/IXI_demograpic_data.csv',sep=',')
df_demographics.rename(columns={"IXI_ID": "subject"}, inplace=True)
df_demographics['subject'] = df_demographics['subject'].apply(lambda x: f'sub-IXI{x}')

In [4]:
df_hists.columns = df_hists.columns.astype(str)
X_hists = list(df_hists.columns)
X_hists = X_hists[1:100]
df_full_histograms = df_hists.merge(df_demographics, on="subject")


In [5]:
df_full_histograms = df_full_histograms.dropna()

In [6]:
creator = PipelineCreator(problem_type="classification")
creator.add("zscore")
creator.add(
    "svm",
    C=(1, 100, "log-uniform")
)

In [7]:
search_params = {
    "kind": "optuna",
    "cv":4
}

scoring = ["balanced_accuracy"]
scores_hists, model_hists, inspector_hists_ridge = run_cross_validation(
    X=X_hists,
    y='SEX_ID (1=m, 2=f)',
    data=df_full_histograms,
    search_params=search_params,
    model=creator,
    return_train_score=True,
    return_inspector=True,
    cv=4,
    scoring = scoring,
)

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/prepare.py:505: RuntimeWarning: The following columns are not defined in X_types: ['Bin_1', 'Bin_2', 'Bin_3', 'Bin_4', 'Bin_5', 'Bin_6', 'Bin_7', 'Bin_8', 'Bin_9', 'Bin_10', 'Bin_11', 'Bin_12', 'Bin_13', 'Bin_14', 'Bin_15', 'Bin_16', 'Bin_17', 'Bin_18', 'Bin_19', 'Bin_20', 'Bin_21', 'Bin_22', 'Bin_23', 'Bin_24', 'Bin_25', 'Bin_26', 'Bin_27', 'Bin_28', 'Bin_29', 'Bin_30', 'Bin_31', 'Bin_32', 'Bin_33', 'Bin_34', 'Bin_35', 'Bin_36', 'Bin_37', 'Bin_38', 'Bin_39', 'Bin_40', 'Bin_41', 'Bin_42', 'Bin_43', 'Bin_44', 'Bin_45', 'Bin_46', 'Bin_47', 'Bin_48', 'Bin_49', 'Bin_50', 'Bin_51', 'Bin_52', 'Bin_53', 'Bin_54', 'Bin_55', 'Bin_56', 'Bin_57', 'Bin_58', 'Bin_59', 'Bin_60', 'Bin_61', 'Bin_62', 'Bin_63', 'Bin_64', 'Bin_65', 'Bin_66', 'Bin_67', 'Bin_68', 'Bin_69', 'Bin_70', 'Bin_71', 'Bin_72', 'Bin_73', 'Bin_74', 'Bin_75', 'Bin_76', 'Bin_77', 'Bin_78', 'Bin_79', 'Bin_80', 'Bin_81', 'Bin_82', 'Bin_83', 'Bin_84', 'Bin_85', 'Bin_86', 'Bin_

In [8]:
scores_hists

,fit_time,score_time,estimator,test_balanced_accuracy,train_balanced_accuracy,n_train,n_test,repeat,fold,cv_mdsum
0,0.492666,0.004918,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.619527,0.820654,378,127,0,0,bc7087515161a73a5a6aff57863f3803
1,0.484841,0.004884,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.687228,0.756322,379,126,0,1,bc7087515161a73a5a6aff57863f3803
2,0.493805,0.004825,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.614902,0.753438,379,126,0,2,bc7087515161a73a5a6aff57863f3803
3,0.514098,0.004676,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.548214,0.881920,379,126,0,3,bc7087515161a73a5a6aff57863f3803


In [10]:
plot_scores(scores_hists)

BokehModel(combine_events=True, render_bundle={'docs_json': {'a2679213-2949-445a-8403-52533fc2346a': {'version…